# Ejercicios
Los datos adjuntos a esta lección forman parte de la base de datos Football Data from Transfermarkt de Kaggle.\
 El conjunto de datos se compone de varios archivos CSV con información sobre competiciones, juegos, clubes, jugadores y apariciones.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import min, max, col
from pyspark.sql.functions import sum, sum_distinct, avg, count
from pyspark.sql.functions import desc, broadcast
# directorio donde estan los datos
path = 'files/football/'

In [2]:
spark = SparkSession.builder.getOrCreate()

24/01/06 18:22:48 WARN Utils: Your hostname, luis-Nitro-AN515-52 resolves to a loopback address: 127.0.1.1; using 192.168.1.17 instead (on interface enp7s0f1)
24/01/06 18:22:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/06 18:22:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Lectura de los archivos
players = spark.read.option('header', 'true').option('inferSchema','true').csv(path+'players.csv')
app = spark.read.option('header', 'true').option('inferSchema','true').csv(path+'appearances.csv')
competicion = spark.read.option('header', 'true').option('inferSchema','true').csv(path+'competitions.csv')
juegos = spark.read.option('header', 'true').option('inferSchema','true').csv(path+'games.csv')

## 1. Determine los tres países con mayor número de jugadores(jugadores nacidos en ese país). El resultado debe estar ordenado de forma descendente.

In [4]:
players.groupBy('country_of_birth').count()\
    .orderBy(desc('count'))\
    .filter(col('country_of_birth').isNotNull())\
    .limit(3).show()

+----------------+-----+
|country_of_birth|count|
+----------------+-----+
|          France| 1694|
|           Spain| 1388|
|           Italy| 1312|
+----------------+-----+



## 2. Obtenga la lista de jugadores con tarjeta roja. La salida debe contener dos columnas, el nombre de pila del jugador y la cantidad de tarjetas rojas que tiene.

In [5]:
tarjetas_roja = app.groupBy('player_id').agg(
    sum('red_cards').alias('tarjetas_rojas')
    ).filter(col('tarjetas_rojas') > 0)
tarjetas_roja.show(5)

+---------+--------------+
|player_id|tarjetas_rojas|
+---------+--------------+
|   199976|             1|
|    57370|             4|
|   172015|             1|
|   253765|             1|
|    18944|             2|
+---------+--------------+
only showing top 5 rows



In [6]:
players_resume = players.select(col('player_id'),col('pretty_name'))
players_resume.show(5)

+---------+-------------------+
|player_id|        pretty_name|
+---------+-------------------+
|    38790|     Dmitri Golubov|
|   106539| Aleksandr Vasiljev|
|   164389|      Rory Donnelly|
|    45247|     Damien Perquis|
|    97205|Nikita Bezlikhotnov|
+---------+-------------------+
only showing top 5 rows



In [7]:
jugadores_tarjetas_rojas = tarjetas_roja.join(players_resume, ['player_id'], 'left')
jugadores_tarjetas_rojas.orderBy(desc('tarjetas_rojas')).select(col('pretty_name'),col('tarjetas_rojas')).show()

+----------------+--------------+
|     pretty_name|tarjetas_rojas|
+----------------+--------------+
|      Thomas Lam|             5|
| Mario Balotelli|             5|
|    Granit Xhaka|             5|
|Domenico Berardi|             5|
|          Hilton|             5|
| Damien Da Silva|             4|
| Yannick Cahuzac|             4|
|     Jaume Costa|             4|
|   Heung Min Son|             4|
| Gabriel Paletta|             4|
|    Jordan Amavi|             4|
|      Ante Rebic|             4|
| Stefan Mitrovic|             4|
| Rodrigo De Paul|             4|
|         Rodrigo|             4|
|     Ivan Ordets|             4|
| Roberto Soriano|             4|
|      Jordi Amat|             4|
|  Ognjen Vranjes|             4|
|      Allan Nyom|             4|
+----------------+--------------+
only showing top 20 rows



## 3. ¿Cuántos juegos se jugaron en la Premier League? La salida debe contener dos columnas, el nombre de la liga y la cantidad de juegos que se jugaron en ella.

In [24]:
premier_league=competicion.filter(col('name')=='premier-league')
premier_league.show()

+--------------+--------------+----------+----------+------------+--------------------+-------------+--------------------+
|competition_id|          name|      type|country_id|country_name|domestic_league_code|confederation|                 url|
+--------------+--------------+----------+----------+------------+--------------------+-------------+--------------------+
|           GB1|premier-league|first_tier|       189|     England|                 GB1|       europa|https://www.trans...|
+--------------+--------------+----------+----------+------------+--------------------+-------------+--------------------+



In [31]:
juegos.filter(col('competition_code')=='GB1')\
    .join(premier_league, col('competition_code') == col('competition_id'), 'left')\
    .groupBy('name').count()\
    .withColumnRenamed('count', 'partidos')\
    .withColumnRenamed('name', 'Liga')\
    .show()

+--------------+--------+
|          Liga|partidos|
+--------------+--------+
|premier-league|    2809|
+--------------+--------+



## 4. Obtenga las tres ligas con mayor número de asistencia de público teniendo en cuenta todos los juegos que se jugaron en ellas. El resultado debe estar ordenado de forma descendente y tener dos columnas, el nombre de la liga y la asistencia total.

In [40]:
juegos.dropDuplicates(['game_id'])\
    .select(col('competition_code'),col('attendance'))\
    .groupBy('competition_code').sum()\
    .join(competicion.select(col('competition_id'),col('name'))
          ,col('competition_code') == col('competition_id'), 'left')\
    .select(col('name').alias('Liga'),col('sum(attendance)').alias('Asistencia_total'))\
    .orderBy(desc('Asistencia_total'))\
    .limit(3)\
    .show()

+--------------+----------------+
|          Liga|Asistencia_total|
+--------------+----------------+
|premier-league|        86964852|
|    bundesliga|        78102473|
|        laliga|        62943533|
+--------------+----------------+



In [ ]:
spark.sparkContext.stop()